### Preprocessing/Analysis (28 pts) ###

In [3]:
import pandas as pd
import numpy as np
ect = pd.read_csv('EuCitiesTemperatures.csv')

In [7]:
for idx in ect.index:
    if(not isinstance(ect.loc[idx, 'country'], str)):
        if (df[df.country == 'HP']['Price'].min() <= df.loc[idx, 'Price'] <= df[df.country == 'HP']['Price'].max()):
            df.loc[idx, 'Category'] = 'HP' 
        
        elif (df[df.country == 'LX']['Price'].min() <= df.loc[idx, 'Price'] <= df[df.country == 'LX']['Price'].max()):
            df.loc[idx, 'Category'] = 'LX'  
            
        elif (df[df.country == 'MR']['Price'].min() <= df.loc[idx, 'Price'] <= df[df.country == 'MR']['Price'].max()):
            df.loc[idx, 'Category'] = 'MR'
            
        elif (df[df.country == 'EC']['Price'].min() <= df.loc[idx, 'Price'] <= df[df.country == 'EC']['Price'].max()):
            df.loc[idx, 'Category'] = 'EC'
            
df

RangeIndex(start=0, stop=213, step=1)

In [10]:
ect.loc[1, 'latitude']

42.5

1. [9 pts] Fill in the missing latitude and longitude values by calculating the average for that country. Round the average to 2 decimal places.

In [2]:
countries = ect['country'].unique()
latlon_list = []
for country in countries:
    lat_avg = round(ect[ect['country']==country]['latitude'].mean() ,2)
    lon_avg = round(ect[ect['country']==country]['longitude'].mean() ,2)
    lat_lon = ect[ect['country']==country]
    lat_lon['latitude'].fillna(lat_avg, inplace=True)
    lat_lon['longitude'].fillna(lon_avg, inplace=True)
    latlon_list.append(lat_lon)
ect = pd.concat(latlon_list)
ect

/Users/LYON/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,city,country,population,EU,coastline,latitude,longitude,temperature
0,Elbasan,Albania,2.90,no,yes,41.12,20.08,15.18
1,Andorra,Andorra,0.07,no,no,42.50,1.52,NaN
2,Innsbruck,Austria,8.57,yes,no,47.85,11.41,4.54
3,Graz,Austria,8.57,yes,no,47.08,14.02,6.91
4,Linz,Austria,8.57,yes,no,48.32,14.29,6.79
...,...,...,...,...,...,...,...,...
208,Edinburgh,United Kingdom,65.11,yes,yes,55.95,-2.94,NaN
209,Exeter,United Kingdom,65.11,yes,yes,50.70,-3.53,10.14
210,Inverness,United Kingdom,65.11,yes,yes,53.55,-4.23,8.00
211,Glasgow,United Kingdom,65.11,yes,yes,55.87,-4.25,8.60


2. [9 pts] Find out the subset of cities that lie between latitudes 40 to 60 (both inclusive) and longitudes 15 to 30 (both inclusive). Find out which countries have the maximum number of cities in this geographical band. (More than one country could have the maximum number of values.)

In [ ]:
sub_citi = ect.loc[(40 <= ect['latitude']) & (ect['latitude'] <= 60) & 
                   (15 <= ect['longitude']) & (ect['longitude'] <= 30)]
sub_citi

In [ ]:
max_citi = sub_citi.groupby('country')['city'].count()
max_citi[max_citi == max(max_citi)]

3. [10 pts] Fill in the missing temperature values by the average temperature value of the similar region type. A region type would be a combinaton of whether it is in EU (yes/no) and whether it has a coastline (yes/no).
For example, if we have a missing temperature value for Bergen, Norway, which is not in the EU but lies on the coast, we will fill it with the average temperature of cities with EU='no' and coastline='yes')

In [ ]:
avg_temp = ect.groupby(['EU', 'coastline'])['temperature'].mean()

content = []
for index in avg_temp.index:
    EU_coa = ect[(ect['EU'] == index[0]) & (ect['coastline'] == index[1])]
    EU_coa['temperature'].fillna(round(avg_temp[index], 2), inplace=True)
    content.append(EU_coa)
ect = pd.concat(content)
ect = ect.sort_index()
ect

### Visualization (27 pts) ###

In [ ]:
import matplotlib.pyplot as plt

1. [6 pts] Plot a bar chart for the number of cities belonging to each of the regions described in Preprocessing/Analysis #3 above.

In [ ]:
ax = ect.groupby(['EU', 'coastline'])['temperature'].count()
plt.bar([str(x) for x in ax.index], ax, color='blue')
plt.xlabel('EU & Coastline', fontsize=14)
plt.ylabel('Number of countries', fontsize=14)

2. [7 pts] Plot a scatter plot of latitude (y-axis) v/s longitude (x-axis) values to get a map-like visual of the cities under consideration. All the cities in the same country should have the same color.

In [ ]:
ect['color'] = 'green'
len(ect['country'].unique())
colors = {}
for i, country in enumerate(ect['country'].unique()):
    colors[country] = i
ect ['color'] = ect['country'].apply(lambda x: colors[x])

plt.scatter(x=ect['longitude'], y=ect['latitude'], c=ect['color'], s=8, alpha=0.5)
plt.xlabel('Longitude', fontsize=14)
plt.ylabel('Latitude', fontsize=14)

3. [6 pts] The population column contains values unique to each country. So two cities of the same country will show the same population value. Plot a histogram of the number of countries belonging to each population group: split the population values into 5 bins (groups).

In [ ]:
ax = ect.groupby('country')['population'].mean()
ax.plot.hist(bins=5, color='red')
plt.xlabel('Population')

4. [8 pts] Plot subplots (2, 2), with proper titles, one each for the region types described in Preprocessing/Analysis #3 above.

Each subplot should be a scatter plot of Latitude (y-axis) vs. City (x-axis), where the color of the plot points should be based on the temperature values: ‘red’ for temperatures above 10, ‘blue’ for temperatures below 6 and ‘orange for temperatures between 6 and 10 (both inclusive). For each subplot, set xticks to an array of numbers from 0 to n-1 (both inclusive), where n is the total number of cities in each region type. This represents each city as a number between 0 and n-1.

In [ ]:
plt.figure(figsize=(16, 10))
def set_color(temperature):
    if temperature > 10:
        return 'red'
    if temperature < 6:
        return 'blue'
    return 'orange'
ect['color'] = ect['temperature'].apply(lambda x: set_color(x))

for i in range(4):
    ect_tmp = ect[(ect['EU']==avg_temp.index[i][0])&(ect['coastline']==avg_temp.index[i][1])]
    ax = plt.subplot(2,2, i+1)
    ax.scatter(ect_tmp['city'], ect_tmp['latitude'],color=ect_tmp['color'])
    ax.set_title('EU={}, Coastline={}'.format(*avg_temp.index[i]))
    ax.set_xticks(range(0,ect_tmp.shape[0]))